In [20]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
from imblearn.under_sampling import TomekLinks 
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import roc_curve, auc
from collections import Counter

In [5]:
#Read the data into dataframe
car_df = pd.read_csv('newCardata.csv')
car_features = pd.read_csv('finalDataPreprocessBinary.csv')
car_label = car_df['FraudFound']



In [13]:
#change the label of the data
labelNo = LabelEncoder()
car_df['FraudFound'] = labelNo.fit_transform(car_df['FraudFound'].astype('str'))
car_label = car_df['FraudFound']
print(car_label.shape)

#split the data set
X_train,X_test,y_train,y_test = train_test_split(car_features,car_label,random_state=3,test_size=0.25)
print('xtrain:',type(X_train))

#split the data into train and test
print(car_features.shape,car_label.shape)


(15419,)
xtrain: <class 'pandas.core.frame.DataFrame'>
(15419, 52) (15419,)


In [15]:
#train the model
model = RandomForestClassifier(random_state=42)
model.fit(X_train,y_train)

print('Random forest classifier:')
predicted = model.predict(X_test)
print(type(predicted),predicted.shape,predicted)
print('Accuracy is ',round(accuracy_score(y_test,model.predict(X_test)) * 100,2))


Random forest classifier:
<class 'numpy.ndarray'> (3855,) [0 0 0 ..., 0 0 0]
Accuracy is  93.59


/usr/local/anaconda3/lib/python3.5/site-packages/sklearn/ensemble/forest.py:248: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [21]:
# calculating specifity and sensitivity
# 0  := Negative
# 1 := Positive
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)
TN, FP, FN, TP = cm.ravel()
print("TN:",TN)
print("FP:",FP)
print("FN:",FN)
print("TP:",TP)

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)

Confusion Matrix:
 [[3603    6]
 [ 241    5]]
TN: 3603
FP: 6
FN: 241
TP: 5
Accuracy: 93.5927367056
Sensitivity: 2.0325203252
Specificity: 99.8337489609


In [22]:

#converting pandas.core.series.Series to numpy.ndarray
print(type(y_test),type(predicted))
ytest = pd.Series(y_test).values
print((ytest.shape[0]))

<class 'pandas.core.series.Series'> <class 'numpy.ndarray'>
3855


In [23]:
#find the index where both are 1.
count = 0
fault = 0
predictedfault = 0
for i in range(predicted.shape[0]):
    if((predicted[i] == 1) and (ytest[i] == 1)):
        count += 1
    if(ytest[i] == 1):
        fault += 1
    if(predicted[i]==1):
        predictedfault += 1
print("No of fraud cases:",fault)
print("No of fauld cases predicted by model:",predictedfault)
print("No of fraud correctly predicted  as fault:",count)

No of fraud cases: 246
No of fauld cases predicted by model: 11
No of fraud correctly predicted  as fault: 5


In [25]:
#with TomekLinks
print('origina dataset shape {}'.format(Counter(car_label)))
tl = TomekLinks(random_state=42)
tl = tl.fit(car_features,car_label)
X_resampled,y_resampled = tl.fit_sample(car_features,car_label)
print('origina dataset shape {}'.format(Counter(y_resampled)))
#print(features.shape,labels.shape)
X_train,X_test,y_train,y_test = train_test_split(X_resampled,y_resampled,random_state=3,test_size=0.25)


print(X_train.shape)

origina dataset shape Counter({0: 14496, 1: 923})
origina dataset shape Counter({0: 14135, 1: 923})
(11293, 52)


In [26]:
print(X_resampled.shape)
print(X_test.shape)

(15058, 52)
(3765, 52)


In [27]:
#model object
model = RandomForestClassifier(n_estimators=100)
model.fit(X_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf='deprecated', min_samples_split=2,
            min_weight_fraction_leaf='deprecated', n_estimators=100,
            n_jobs=None, oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [28]:
print('Random forest classifier:')
predicted = model.predict(X_test)
print(type(predicted),predicted.shape,predicted)
print('Accuracy is ',round(accuracy_score(y_test,model.predict(X_test)) * 100,2))


Random forest classifier:
<class 'numpy.ndarray'> (3765,) [0 0 0 ..., 0 0 0]
Accuracy is  93.55


In [29]:
cm = confusion_matrix(y_test,predicted)
print("Confusion Matrix:\n",cm)
TN, FP, FN, TP = cm.ravel()

print("Accuracy:",(TP+TN)/(TP+FP+FN+TN)*100)
print("Sensitivity:",TP/(TP+FN)*100)
print("Specificity:",TN/(TN+FP)*100)

Confusion Matrix:
 [[3516    1]
 [ 242    6]]
Accuracy: 93.5458167331
Sensitivity: 2.41935483871
Specificity: 99.9715666761


In [22]:
#converting pandas.core.series.Series to numpy.ndarray
ytest = pd.Series(y_test).values
print((ytest.shape[0]))
print(predicted.shape[0])
print(ytest)
print()

3765
3765
[1 0 0 ..., 0 0 0]



In [30]:
#find the index where both are 1.
count = 0
fault = 0
predictedfault = 0
for i in range(predicted.shape[0]):
    
    if((predicted[i] == 1) and (ytest[i] == 1)):
        count += 1
    if(ytest[i] == 1):
        fault += 1
    if(predicted[i]==1):
        predictedfault += 1
print("No of fraud cases:",fault)
print("No of fauld cases predicted by model:",predictedfault)
print("No of fraud correctly predicted  as fault:",count)

No of fraud cases: 239
No of fauld cases predicted by model: 7
No of fraud correctly predicted  as fault: 0


In [31]:
from sklearn.metrics import precision_recall_fscore_support

precision_recall_fscore_support(y_test, predicted, average='micro')
precision_recall_fscore_support(y_test, predicted, average='macro')
precision_recall_fscore_support(y_test, predicted, average='weighted')


(0.93043581774172979, 0.93545816733067733, 0.90602803099306828, None)

In [33]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn import metrics
from sklearn import model_selection
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score



print('Random forest classifier:')

tl = TomekLinks(random_state=42)
X_resampled,Y_resampled = tl.fit_sample(car_features,car_label)
print('Resampled dataset shape {}'.format(Counter(Y_resampled)))

#convert np.array to dataframe 
df_features = pd.DataFrame(X_resampled)
df_lables = pd.DataFrame(Y_resampled)



Random forest classifier:
Resampled dataset shape Counter({0: 14135, 1: 923})


In [34]:
#add the both the labels and features
df_features['FraudFound']= df_lables

#shuffle the featrues
new_combine_features = df_features.set_index(np.random.permutation(df_features.index))


new_label = new_combine_features['FraudFound']

#drop the FraudFound attribute
new_combine_features.drop(['FraudFound'],inplace=True,axis=1)

scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}

kfold = model_selection.KFold(n_splits=10, random_state=10)
model=RandomForestClassifier(n_estimators=100) 

results = model_selection.cross_validate(estimator=model,X=new_combine_features,y=new_label,cv=kfold,scoring=scoring)

print(np.mean(results['test_accuracy']))
print(np.mean(results['test_precision']))
print(np.mean(results['test_recall']))
print(np.mean(results['test_f1_score']))



0.940232116936
0.696666666667
0.0322235879308
0.0614355300258
